In [1]:
# Drive mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# GPU Check
!nvidia-smi

Sun Feb  2 09:32:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#####################################
# Data Set                          #
#####################################
import os
import glob
from datetime import date
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

import imgaug as ia
from imgaug import augmenters as iaa

class FaceDataset(Dataset):
  def __init__(self, train_table, train_dir, transform=None):
    self.train_dir = train_dir
    self.train_table = train_table
    
    self.H = 128 # Height
    self.W = 128 # Width
    self.transform = transform
      
  def __len__(self):
    return len(self.train_table)
  
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist() # to list
    
    filename = self.train_table['filename'][idx]
    img_file = self.train_dir + filename
    try:
      img = cv2.imread(img_file)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      label = self.train_table['label'][idx]
    except:
      # Error Save
      today = date.today().strftime("%y%m%d")
      if not os.path.isdir('drive/My Drive/team10/errors/' + today):
        os.makedirs('drive/My Drive/team10/errors/' + today)
      error_file = 'drive/My Drive/team10/errors/' + today + '/' + today + '.txt'
      with open(error_file, 'a') as f:
        f.write(filename + '\n')
      
      filename = self.train_table['filename'][0]
      img_file = self.train_dir + filename
      img = cv2.imread(img_file)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      label = self.train_table['label'][0]
    
    sample = {'img' : img, 'label': label}
    if self.transform:
      sample = self.transform(sample)

    # numpy array to tesnor
    img = torch.tensor(sample['img'].transpose((2, 0, 1)))
    label = torch.tensor(sample['label'])
    sample = {'img' : img, 'label': label}

    return sample



#####################################
# Data Loader                       #
#####################################

from torch.utils.data import DataLoader

root = 'drive/My Drive/team10'
train_dir = root + '/face_images_128x128/'
train_csv = glob.glob(root + '/csvs/*train*')[0]

# Split train and validation data
train_table = pd.read_csv(train_csv)
valid_table = train_table.sample(frac=0.2, random_state=999)
# train_table.drop(index=valid_table.index, axis=0, inplace=True)

# Reset index (0, 1, 2, ...)
train_table.reset_index(inplace=True)
valid_table.reset_index(inplace=True)

BATCH_SIZE = 16

train_dataset = FaceDataset(train_table=train_table, train_dir=train_dir)
valid_dataset = FaceDataset(train_table=valid_table, train_dir=train_dir)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

batch = 0
cnt = 0
for i in train_loader:
  print(batch)
  batch += 1
  for j in range(BATCH_SIZE):
    img = i['img'][j]
    label = i['label'][j]
    cnt += 1
    print(cnt, np.shape(img), label)
  break  

In [4]:
#####################################
# Model, Loss, Optimizer            #
#####################################
import torch
import torch.nn as nn
import torchvision

N_CLASS = 6
LEARNING_RATE = 1e-4

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Select model
# 'resnet18', 'resnet50', 'resnet101', 'resnet152'
model_name = 'resnet18'
model = torch.hub.load('pytorch/vision:v0.5.0', model_name, pretrained=False)
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=N_CLASS, bias=True)
model = model.to(device)

crossEntropyLoss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

cuda:0


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.5.0


In [8]:
#####################################
# Settings for saving model, board  #
#####################################
from datetime import date
from torch.utils.tensorboard import SummaryWriter

N_EPOCH = 100

# Model save name
today = date.today().strftime("%y%m%d")
weight_name = today + \
              '_' + model_name + \
              '_batch' + str(BATCH_SIZE) + \
              '_lr' + str(LEARNING_RATE).split('.')[-1] + \
              '_epoch' + str(N_EPOCH)
if not os.path.isdir(root + '/weights/' + today):
  os.makedirs(root + '/weights/' + today)
weight_savename = root + '/weights/' + today + '/' + weight_name + '.pth'

# Tensorboard for logging
if not os.path.isdir(root + '/tensorboard/' + today):
  os.makedirs(root + '/tensorboard/' + today)
writer = SummaryWriter(root + '/tensorboard/' + today + '/' + weight_name)


#####################################
# Train                             #
#####################################
from tqdm import tqdm

min_loss = float('inf')
for ith, epoch in enumerate(range(N_EPOCH), start=1):
  loss_per_epoch = 0.0
  
  for batch in tqdm(train_loader):
    imgs = batch['img'].to(device, dtype=torch.float)
    labels = batch['label'].to(device)

    optimizer.zero_grad()

    outs = model(imgs)
    _, pred = torch.max(outs, 1)
    loss = crossEntropyLoss(outs, labels)
    loss.backward()
    optimizer.step()
    loss_per_epoch += loss.item()
    
    if loss_per_epoch < min_loss:
        torch.save(model, weight_savename)
        min_loss = loss_per_epoch
          
  loss_per_epoch /= len(train_loader)
  print('epoch: {}, train loss: {}'.format(epoch + 1, loss_per_epoch))
  # Log on the tensorboard
  writer.add_scalar('training loss',
                      loss_per_epoch,
                      epoch + 1)

  0%|          | 0/344 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be 

epoch: 1, train loss: 0.6378904349755409


  0%|          | 0/344 [00:00<?, ?it/s]

epoch: 2, train loss: 0.279610245943416


  0%|          | 0/344 [00:00<?, ?it/s]

...

epoch: 98, train loss: 0.014303279416158466


  0%|          | 1/344 [00:00<00:35,  9.69it/s]

epoch: 99, train loss: 0.003323127201563397


100%|██████████| 344/344 [00:30<00:00, 11.68it/s]

epoch: 100, train loss: 0.0010152444272027298


In [3]:
errors

NameError: ignored